# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [42]:
!pip install python-dotenv

PRICE_DATA=('c:/Users/Anca/Desktop/UofT DSI/Production/05_src/data/prices')

from dotenv import load_dotenv
import os

load_dotenv()

price_data = os.getenv(PRICE_DATA)

df = dd.read_parquet('c:/Users/Anca/Desktop/UofT DSI/Production/05_src/data/prices')


In [35]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [45]:
# Write your code below.

from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join('c:/Users/Anca/Desktop/UofT DSI/Production/05_src/data/prices'))
parquet_files
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


c:\Users\Anca\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [52]:
# Write your code below.

import numpy as np

dd_rets = (dd_px.groupby('ticker', group_keys=False)
           .apply(lambda x: x.assign(Close_lag_1=x['Close'].shift(1),
                                     Adj_Close_lag_1=x['Adj Close'].shift(1)))
           .assign(returns=lambda x: x['Adj Close'] / x['Adj_Close_lag_1'] - 1,
                   high_low_range=lambda x: x['High'] - x['Low']))

dd_feat = dd_rets

print(dd_feat.head)


<bound method _Frame.head of Dask DataFrame Structure:
                             Date     Open     High      Low    Close Adj Close Volume  sector subsector   year Close_lag_1 Adj_Close_lag_1  returns high_low_range
npartitions=11132                                                                                                                                                  
                   datetime64[ns]  float64  float64  float64  float64   float64  int64  object    object  int32     float64         float64  float64        float64
                              ...      ...      ...      ...      ...       ...    ...     ...       ...    ...         ...             ...      ...            ...
...                           ...      ...      ...      ...      ...       ...    ...     ...       ...    ...         ...             ...      ...            ...
                              ...      ...      ...      ...      ...       ...    ...     ...       ...    ...         ...  

C:\Users\Anca\AppData\Local\Temp\ipykernel_23016\925717860.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_rets = (dd_px.groupby('ticker', group_keys=False)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [55]:
# Write your code below.

import pandas as pd 
df_pandas = dd_feat.compute()

df_pandas['rolling_avg_return'] = df_pandas['returns'].rolling(window=10).mean()

print(df_pandas.head())


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
RVTY   2008-01-02  25.799999  26.010000  25.240000  25.530001  22.805971   
RVTY   2008-01-03  25.790001  25.790001  25.059999  25.180000  22.493313   
RVTY   2008-01-04  24.870001  24.990000  24.430000  24.490000  21.876934   
RVTY   2008-01-07  24.590000  24.850000  24.450001  24.780001  22.136000   
RVTY   2008-01-08  24.700001  25.190001  24.700001  24.879999  22.225328   

         Volume       sector              subsector  year  Close_lag_1  \
ticker                                                                   
RVTY    1253600  Health Care  Health Care Equipment  2008          NaN   
RVTY    1671500  Health Care  Health Care Equipment  2008    25.530001   
RVTY    1657800  Health Care  Health Care Equipment  2008    25.180000   
RVTY    1553500  Health Care  Health Care Equipment  2008    24.490000   
RVTY    1580100  Health

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
The calculation was done in Pandas due to the size of the dataset and the fact that it's done locally without parallel computing. 
It could have been done in Dask as well; however, for the purpose of this analysis, Pandas' straightforward model makes it a better choice. 


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.